In [ ]:
import helper.utils
import helper
from helper import rolling_window, shade_area, utils, generate_shadowed_img
import preprocessing.shadow_reduction
import cv2
import time
import os
import numpy as np

TRAIN_DATA = 'C:\\Users\\Dominic\\Desktop\\Academics\\Term 7\\Computer Vision\\dump\\Concrete Crack Images for Classification'

IN_VIDEO = "C:\\Users\\Dominic\\Desktop\\Academics\\Term 7\\Computer Vision\\dump\\cnn out\\File_003.mov"
IN_FRAMES = "C:\\Users\\Dominic\\Desktop\\Academics\\Term 7\\Computer Vision\\dump\\test" # "F://term7//CV//ProjectTrail//IMG_5175frame"

OUT_FRAMES =  "C:\\Users\\Dominic\\Desktop\\Academics\\Term 7\\Computer Vision\\dump\\out_test\\"
OUT_VIDEO = "C:\\Users\\Dominic\\Desktop\\Academics\\Term 7\\Computer Vision\\dump\\test.mov"

# automatically reload imported modules
%load_ext autoreload
%autoreload 2

# Load Data

In [ ]:
train_size = 200
test_size = 100
val_size = 40

img_range = np.arange(1, 800)
X_train_pos_idx, X_test_pos_idx, X_val_pos_idx = helper.get_random_indices(img_range, train_size, test_size, val_size)
X_train_neg_idx, X_test_neg_idx, X_val_neg_idx = helper.get_random_indices(img_range, train_size, test_size, val_size)

X_train, Y_train = helper.get_concrete_data(X_train_pos_idx, X_train_neg_idx, path = TRAIN_DATA)
#X_test , Y_test  = helper.get_concrete_data(X_test_pos_idx, X_test_neg_idx, path = TRAIN_DATA)
#X_val  , Y_val   = helper.get_concrete_data(X_val_pos_idx, X_val_neg_idx, path = TRAIN_DATA)

print( X_train.shape, Y_train.shape )
#print( X_test.shape , Y_test.shape  )
#print( X_val.shape  , Y_val.shape   )

# Load Filters

In [ ]:
from filters import *
from preprocessing import shadow_reduction
bilateral_canny = BilateralCanny()

filters = {}
filters['canny']           = bilateral_canny.canny_img
filters['bilateral canny'] = bilateral_canny.bilateral_canny_img

# Train Model

In [ ]:
from sklearn import linear_model

X_train = np.array(list(shadow_reduction.adaptiveThreshold(x) for x in X_train))
tmpX = np.array(list(filters['bilateral canny'](x) for x in X_train)).reshape(2*train_size, -1)
model =  linear_model.LogisticRegression(C=1e5)
model.fit(tmpX, Y_train)

# Video to Frames

In [ ]:
from video import video
v = video(IN_VIDEO)
v.video_to_frames(IN_FRAMES)

### process to video

## CAREFUL!
in the cell below:         frame_final = generate_shadowed_img(frame, frame_after_canny, model)

in the function generate_shadowed_img, model.predict(img) will be called. So for CNN check the predict function again!!!!!

In [ ]:
input_format = 'jpg'

bilateral_canny_video = filters['canny']

time_start = time.time()
time_pre = time.time()
time_norm = 0
time_canny =  0
time_predict_and_shadow = 0
time_save = 0

count = 0
frameid = 0
timelog = True

for f in os.listdir(IN_FRAMES):
    if f.endswith(input_format):
        image_path = os.path.join(IN_FRAMES, f)
        frame = cv2.imread(image_path)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if timelog:
            time_pre = time.time()
        
        # shadow reduction
        frame_shadow_reduced = shadow_reduction.adaptiveThreshold(gray_frame)
        
        if timelog:
            time_norm += time.time() - time_pre
            time_pre = time.time()
        
            
        # generate output after canny filter, or tophat
        frame_after_canny = bilateral_canny_video(frame_shadow_reduced)
        
        if timelog:
            time_canny += time.time() - time_pre
            time_pre = time.time()    
        
        # shadowed image
        frame_final = generate_shadowed_img(frame, frame_after_canny, model)
        
        if timelog:
            time_predict_and_shadow += time.time() - time_pre
            time_pre = time.time()
        
        # save to folder
        utils.save_image(cv2.cvtColor(frame_final, cv2.COLOR_RGB2BGR), OUT_FRAMES, f)
        if timelog:
            time_save = time.time() - time_pre
            time_pre  = time.time()    
        
if timelog:
    print("total time for frames generation is {}".format(time.time()-time_start))
    print("time norm               : {}".format(time_norm))
    print("time_canny              : {}".format(time_canny))
    print("time_predict_and_shadow : {}".format(time_predict_and_shadow))
    print("time_save               : {}".format(time_save))

### output to video

In [ ]:
time_start = time.time()
from video import video
v = video(IN_VIDEO)
v.frames_to_video(OUT_VIDEO, input_loc=OUT_FRAMES, debug=True)
print("total time video saving {}".format(time.time()-time_start))